# Wishing Happy New Year 2024 Python code

# Code_With_Chinmayee Nayak

# Explanation


This code is a Python program that uses the Pygame library to create a fireworks display and other animated elements with dispalying the message Happy New year

# Initialization:

Pygame is initialized.
Constants like WIDTH, HEIGHT, FPS, and COLORS are defined.
A Pygame window is created with the specified dimensions.

In [6]:
import pygame
import time
import random
import math

pygame.init()

WIDTH, HEIGHT = 800, 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Fireworks!")

FPS = 60

COLORS = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (0, 255, 255),
    (255, 165, 0),
    (255, 255, 255),
    (230, 230, 250),
    (255, 192, 203)
]

#Projectile Class:
#Represents a single projectile (particle) in the fireworks.
#It has methods to move and draw the projectile with a decaying alpha value for a fading effect.

class Projectile:
    WIDTH = 5
    HEIGHT = 10
    ALPHA_DECREMENT = 3

    def __init__(self, x, y, x_vel, y_vel, color):
        self.x = x
        self.y = y
        self.x_vel = x_vel
        self.y_vel = y_vel
        self.color = color
        self.alpha = 255

    def move(self):
        self.x += self.x_vel
        self.y += self.y_vel
        self.alpha = max(0, self.alpha - self.ALPHA_DECREMENT)

    def draw(self, win):
        self.draw_rect_alpha(win, self.color + (self.alpha,), (self.x, self.y, self.WIDTH, self.HEIGHT))

    @staticmethod
    def draw_rect_alpha(surface, color, rect):
        shape_surf = pygame.Surface(pygame.Rect(rect).size, pygame.SRCALPHA)
        pygame.draw.rect(shape_surf, color, shape_surf.get_rect())
        surface.blit(shape_surf, rect)
        
# Firework Class:
# Represents a firework that explodes into multiple projectiles.
# Two types of explosions are supported: circular and star-shaped.
# Methods include explode, create_circular_projectiles, create_star_projectiles, move, and draw.

class Firework:
    RADIUS = 10
    MAX_PROJECTILES = 50
    MIN_PROJECTILES = 25
    PROJECTILE_VEL = 4

    def __init__(self, x, y, y_vel, explode_height, color):
        self.x = x
        self.y = y
        self.y_vel = y_vel
        self.explode_height = explode_height
        self.color = color
        self.projectiles = []
        self.exploded = False

    def explode(self):
        self.exploded = True
        num_projectiles = random.randrange(self.MIN_PROJECTILES, self.MAX_PROJECTILES)

        if random.randint(0, 1) == 0:
            self.create_circular_projectiles(num_projectiles)
        else:
            self.create_star_projectiles()

    def create_circular_projectiles(self, num_projectiles):
        angle_dif = math.pi * 2 / num_projectiles
        current_angle = 0
        vel = random.randrange(self.PROJECTILE_VEL - 1, self.PROJECTILE_VEL + 1)

        for __ in range(num_projectiles):
            x_vel = math.sin(current_angle) * vel
            y_vel = math.cos(current_angle) * vel
            color = random.choice(COLORS)
            self.projectiles.append(Projectile(self.x, self.y, x_vel, y_vel, color))
            current_angle += angle_dif

    def create_star_projectiles(self):
        angle_diff = math.pi / 4
        current_angle = 0
        num_projectiles = 32

        for i in range(1, num_projectiles + 1):
            vel = self.PROJECTILE_VEL + (i % (num_projectiles / 8))
            x_vel = math.sin(current_angle) * vel
            y_vel = math.cos(current_angle) * vel
            color = random.choice(COLORS)
            self.projectiles.append(Projectile(self.x, self.y, x_vel, y_vel, color))

            if i % (num_projectiles / 8) == 0:
                current_angle += angle_diff

    def move(self, max_width, max_height):
        if not self.exploded:
            self.y += self.y_vel
            if self.y <= self.explode_height:
                self.explode()

        projectiles_to_remove = []
        for projectile in self.projectiles:
            projectile.move()

            if not (0 <= projectile.x < max_width and 0 <= projectile.y < max_height):
                projectiles_to_remove.append(projectile)

        for projectile in projectiles_to_remove:
            self.projectiles.remove(projectile)

    def draw(self, win):
        if not self.exploded:
            pygame.draw.circle(win, self.color, (self.x, self.y), self.RADIUS)

        for projectile in self.projectiles:
            projectile.draw(win)
            
# TextAnimation Class:
# Represents an animated text on the screen with optional blinking.
# Uses an emoji font for rendering.

class TextAnimation:
    def __init__(self, text, x, y, start_time, duration, color=(255, 255, 255), bold=False, italic=False, blink_frequency=5):
        self.text = text
        self.x = x
        self.y = y
        self.start_time = start_time
        self.duration = duration
        self.color = color
        self.bold = bold
        self.italic = italic
        self.blink_frequency = blink_frequency
        self.emoji_font = pygame.font.Font("C:\\Users\\nchin\\Desktop\\seguiemj.ttf", 36)  # Replace with the path to your emoji font

    def draw(self, win, current_time):
        elapsed_time = current_time - self.start_time

        if int(elapsed_time * self.blink_frequency) % 2 == 0:
            alpha = 255
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        else:
            alpha = 0
            color = (255, 255, 255)

        text_surface = self.emoji_font.render(self.text, True, (*color, alpha))
        rect = text_surface.get_rect()
        rect.center = (self.x, self.y)
        win.blit(text_surface, rect)
        
# Star Class:
# Represents a star with a changing alpha value for twinkling effect.

class Star:
    def __init__(self, x, y, radius, color):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.alpha = random.randint(100, 255)
        self.alpha_increment = random.randint(1, 3)

    def move(self):
        self.alpha += self.alpha_increment
        if self.alpha > 255 or self.alpha < 100:
            self.alpha_increment *= -1

    def draw(self, win):
        alpha = max(0, min(255, self.alpha))
        star_surface = pygame.Surface((self.radius * 2, self.radius * 2), pygame.SRCALPHA)
        pygame.draw.circle(star_surface, self.color + (alpha,), (self.radius, self.radius), self.radius)
        win.blit(star_surface, (self.x - self.radius, self.y - self.radius))
        
# Launcher Class:
# Represents a launcher that launches fireworks at a specified frequency.
# Uses the Firework class.

class Launcher:
    WIDTH = 20
    HEIGHT = 20
    COLOR = 'grey'

    def __init__(self, x, y, frequency):
        self.x = x
        self.y = y
        self.frequency = frequency
        self.start_time = time.time()
        self.fireworks = []

    def draw(self, win):
        pygame.draw.rect(win, self.COLOR, (self.x, self.y, self.WIDTH, self.HEIGHT))

        for firework in self.fireworks:
            firework.draw(win)

    def launch(self):
        color = random.choice(COLORS)
        explode_height = random.randrange(50, 400)
        firework = Firework(self.x + self.WIDTH/2, self.y, -5, explode_height, color)
        self.fireworks.append(firework)

    def loop(self, max_width, max_height):
        current_time = time.time()
        time_elapsed = current_time - self.start_time

        if time_elapsed * 1000 >= self.frequency:
            self.start_time = current_time
            self.launch()

        fireworks_to_remove = []
        for firework in self.fireworks:
            firework.move(max_width, max_height)
            if firework.exploded and len(firework.projectiles) == 0:
                fireworks_to_remove.append(firework)

        for firework in fireworks_to_remove:
            self.fireworks.remove(firework)
            
# AnimatedBalloon Class:
# Represents a balloon that moves upwards on the screen.

class AnimatedBalloon:
    def __init__(self, x, y, radius, color, x_vel, y_vel):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.x_vel = x_vel
        self.y_vel = y_vel

    def move(self, max_width, max_height):
        self.x += self.x_vel
        self.y += self.y_vel

        if self.y < 0 or self.y > max_height or self.x < 0 or self.x > max_width:
            self.x = random.randint(0, max_width)
            self.y = max_height

    def draw(self, win):
        pygame.draw.circle(win, self.color, (int(self.x), int(self.y)), self.radius)
        
# DigitalClock Class:
# Represents a digital clock that displays the current time.

class DigitalClock:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.radius = 30
        self.font = pygame.font.SysFont(None, 36)

    def draw(self, win, current_time):
        time_str = time.strftime("%H:%M:%S", time.localtime(current_time))

        pygame.draw.circle(win, (0, 0, 0), (self.x, self.y), self.radius)
        text_surface = self.font.render(time_str, True, (255, 255, 255))
        rect = text_surface.get_rect()
        rect.center = (self.x, self.y)
        win.blit(text_surface, rect)
        
# Draw Functions:
# draw: Clears the screen, updates and draws stars, launchers, balloons, digital clock, text animations, and clock hands.
# draw_clock_hands: Draws clock hands based on the current time.
# draw_clock_hand: Draws a single clock hand.
# draw_clock_numbers: Draws numbers around the clock.

def draw(launchers, text_animations, stars, balloons, digital_clock, current_time):
    win.fill((0, 0, 0))

    for star in stars:
        star.move()
        star.draw(win)

    for launcher in launchers:
        launcher.draw(win)

    for balloon in balloons:
        balloon.move(WIDTH, HEIGHT)
        balloon.draw(win)

    digital_clock.draw(win, current_time)

    for text_animation in text_animations:
        text_animation.draw(win, current_time)

    draw_clock_hands(win, current_time, WIDTH // 2, HEIGHT // 2)

    pygame.display.flip()

def draw_clock_hands(win, current_time, center_x, center_y):
    hours, minutes, seconds = get_current_time(current_time)

    hour_angle = math.radians((hours % 12) * 30 + minutes / 2)
    draw_clock_hand(win, center_x, center_y, hour_angle, 50, 5)
    draw_clock_numbers(win, center_x, center_y, 50, hour_angle, 12)

    minute_angle = math.radians(minutes * 6 + seconds / 10)
    draw_clock_hand(win, center_x, center_y, minute_angle, 80, 3)
    draw_clock_numbers(win, center_x, center_y, 80, minute_angle, 60)

    second_angle = math.radians(seconds * 6)
    draw_clock_hand(win, center_x, center_y, second_angle, 90, 1, (255, 0, 0))

def draw_clock_hand(win, center_x, center_y, angle, length, width, color=(255, 255, 255)):
    x = center_x + length * math.cos(angle)
    y = center_y + length * math.sin(angle)
    pygame.draw.line(win, color, (center_x, center_y), (x, y), width)

def draw_clock_numbers(win, center_x, center_y, length, angle, count):
    font = pygame.font.SysFont(None, 24)
    for i in range(1, count + 1):
        num_x = center_x + length * math.cos(angle)
        num_y = center_y + length * math.sin(angle)
        num_text = font.render(str(i), True, (255, 255, 255))
        num_rect = num_text.get_rect(center=(num_x, num_y))
        win.blit(num_text, num_rect)
        
# Clock-related Functions:
# get_current_time: Returns the current time as hours, minutes, and seconds.
# main: Main program loop where Pygame events are handled, launchers are updated, and the screen is updated and rendered.

def get_current_time(current_time):
    current_struct_time = time.localtime(current_time)
    return current_struct_time.tm_hour, current_struct_time.tm_min, current_struct_time.tm_sec

def main():
    run = True
    clock = pygame.time.Clock()

    launchers = [
        Launcher(100, HEIGHT - Launcher.HEIGHT, 3000),
        Launcher(300, HEIGHT - Launcher.HEIGHT, 4000),
        Launcher(500, HEIGHT - Launcher.HEIGHT, 2000),
        Launcher(700, HEIGHT - Launcher.HEIGHT, 5000)
    ]

    text_animations = [
        TextAnimation(" 🥂҉!!Welcome!!🥂҉", WIDTH // 2, HEIGHT // 2 + 50, time.time(), 5,
                       color=(255, 0, 0), bold=True, italic=True, blink_frequency=5),
        TextAnimation("🎈🌟🎄🌹Happy New Year 2024🌹🎄🌟🎈", WIDTH // 2, HEIGHT // 2 + 100, time.time() + 5, 5,
                       color=(255, 0, 0), bold=True, italic=True, blink_frequency=5)
    ]

    stars = [Star(random.randint(0, WIDTH), random.randint(0, HEIGHT), random.randint(1, 3), (255, 255, 255)) for _ in range(100)]

    balloons = [
        AnimatedBalloon(random.randint(0, WIDTH), HEIGHT, random.randint(10, 30), random.choice(COLORS),
                        random.uniform(-1, 1), random.uniform(-0.5, -1)) for _ in range(20)
    ]

    digital_clock = DigitalClock(WIDTH // 2, HEIGHT // 2)

    while run:
        clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break

        current_time = time.time()

        for launcher in launchers:
            launcher.loop(WIDTH, HEIGHT)

        if time.localtime(current_time).tm_hour == 0 and time.localtime(current_time).tm_min == 0 and time.localtime(current_time).tm_sec == 0:
            print("Happy New Year!")

        draw(launchers, text_animations, stars, balloons, digital_clock, current_time)

        pygame.display.flip()

    pygame.quit()
    
# Execution:
# The program enters the main loop (main) where Pygame events are handled, launchers are updated, and the screen is updated and rendered.

if __name__ == '__main__':
    main()

# Conclusion

The program creates a Pygame window and animates a New Year's Eve celebration with fireworks, balloons, stars,Emoji and a digital clock. The various classes and functions work together to create a visually appealing and dynamic display